## Note: 
Only run the cells below if you need to either build the sqlite database from scratch, or if you need to download new data. If you are downloading new data, make sure to put the database in the folder 'zip_data'.

If you don't have the database you need, go to https://drive.google.com/file/d/1J1TLVkvnEQvX31MxdF2df04BNpiaExyf/view?usp=sharing and then place it in the zip_data folder.

Don't forget to replace the link above with a new one that has the updated database!

Also be warned that constructing the database from scratch will take several hours!

In [1]:
import requests as r
import pandas as pd
import datetime
import requests as r
import db_zip_populator as zp
import re
import sqlite3
from tqdm import tqdm
import re
import duckdb

endyear = datetime.date.today().year
api_headers = {}
api_headers['x-api-key'] = '975f39a54e48438ceebf303d6018e34db212e804'
db_path = '../zip_data/duck_db_manager/database/us_economic_data.duckdb'


# Zipcode data


- The ZipCodeUtility class has 'startyear' and 'endyear' as parameters. If you need to download new data, you can adjust the start and end years. 

- There is a possibility that this won't work if the api call structure is changed. If that's the case, adjusting the '_get_response_data' function in db_zip_populator.py will do the trick.

In [13]:
# If you need to update the database, use the 'startyear' and 'endyear' parameters
# For example, setting startyear and endyear to 2024 will get data for 2024
# zip_util = zu.ZipCodeUtility(api_headers=api_headers, startyear=2024, endyear=2024)
zip_populator = zp.ZipPopulator(api_headers=api_headers, startyear=2012, endyear=2023)

In [3]:
zip_populator.endyear = 2019

In [4]:
zip_populator.get_zip_for_year(2020)

Inserting for year: 2020: 100%|██████████| 1403/1403 [23:04<00:00,  1.01it/s] 


In [3]:
for i in range(2012, 2019):
     zip_populator.get_zip_for_year(i)

Inserting for year: 2018: 100%|██████████| 1403/1403 [22:20<00:00,  1.05it/s] 


In [3]:
zip_populator.get_zip_for_year(2019)

Inserting for year: 2019: 100%|██████████| 1403/1403 [23:01<00:00,  1.02it/s] 


In [12]:
for i in range(2020, 2022):
     zip_populator.get_zip_for_year(i)

Inserting for year: 2021: 100%|██████████| 1403/1403 [33:39<00:00,  1.44s/it] 


In [15]:
zip_populator.db_populator.open()
zip_populator.get_zip_zbp('00')
zip_populator.db_populator.close()


In [ ]:
conn = duckdb.connect(db_path)
schema_query = "DESCRIBE DataEntry;"
result = conn.execute(schema_query).fetchall()
print(result)


# Check

## The cells below check if the database was populated properly

In [8]:
!lsof ./zip_data/us_economic_data.duckdb


COMMAND     PID  USER   FD   TYPE DEVICE  SIZE/OFF    NODE NAME
python3.9 86202 David   73u   REG   1,13 174600192 6742570 ./zip_data/us_economic_data.duckdb


In [9]:
!kill -9 86202


In [24]:
conn = duckdb.connect(db_path)
# Execute the query to get all tables
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

# Print the list of tables
print(tables)
conn.close()

[('DataEntry',), ('DimNaics',), ('DimYear',), ('DimZipCode',)]


In [26]:
conn = duckdb.connect(db_path)
query = """
    SELECT 
        DataEntry.*, 
        DimZipCode.City, 
        DimZipCode.State 
    FROM 
        DataEntry  
    JOIN 
        DimZipCode 
    ON 
        DataEntry.GeoID = DimZipCode.GeoID
    WHERE 
        DataEntry.GeoID = '21043'
"""
conn.execute(query)
rows = conn.fetchall()
columns = [description[0] for description in conn.description]
print(columns)
for row in rows:
    print(row)
len(rows)
conn.close()

['EntryID', 'GeoID', 'NaicsCode', 'Year', 'Establishments', 'Employees', 'Payroll', 'IndustryLevel', 'City', 'State']
(374897, '21043', '4451', 2020, 10, 0, 0, 4, 'Ellicott City', 'MD')
(383373, '21043', '445110', 2020, 6, 0, 0, 6, 'Ellicott City', 'MD')
(389955, '21043', '445120', 2020, 4, 0, 0, 6, 'Ellicott City', 'MD')
(398207, '21043', '4453', 2020, 6, 0, 0, 4, 'Ellicott City', 'MD')
(402834, '21043', '445310', 2020, 6, 0, 0, 6, 'Ellicott City', 'MD')
(407740, '21043', '4461', 2020, 8, 0, 0, 4, 'Ellicott City', 'MD')
(415920, '21043', '446110', 2020, 5, 0, 0, 6, 'Ellicott City', 'MD')
(428918, '21043', '4471', 2020, 11, 0, 0, 4, 'Ellicott City', 'MD')
(440822, '21043', '447110', 2020, 6, 0, 0, 6, 'Ellicott City', 'MD')
(450625, '21043', '447190', 2020, 5, 0, 0, 6, 'Ellicott City', 'MD')
(452445, '21043', '4481', 2020, 5, 0, 0, 4, 'Ellicott City', 'MD')
(477283, '21043', '4511', 2020, 6, 0, 0, 4, 'Ellicott City', 'MD')
(483654, '21043', '451120', 2020, 3, 0, 0, 6, 'Ellicott City', '

In [4]:
conn = duckdb.connect(db_path)
query = "SELECT * FROM DimZipcode"
conn.execute(query)
rows = conn.fetchall()
columns = [description[0] for description in conn.description]
print(columns)
for row in rows:
    print(row)
len(rows)

['GeoID', 'City', 'State']
('86433', 'Oatman', 'AZ')
('25161', 'Powellton', 'WV')
('67640', 'Gorham', 'KS')
('37371', 'Athens', 'TN')
('56134', 'Hardwick', 'MN')
('16427', 'Mill Village', 'PA')
('52362', 'Wyoming', 'IA')
('28746', 'Lake Lure', 'NC')
('23407', 'Mappsville', 'VA')
('76021', 'Bedford', 'TX')
('77043', 'Houston', 'TX')
('98165', 'Seattle', 'WA')
('74941', 'Keota', 'OK')
('29039', 'Cordova', 'SC')
('23507', 'Norfolk', 'VA')
('67583', 'Turon', 'KS')
('70392', 'Patterson', 'LA')
('23009', 'Aylett', 'VA')
('29429', 'Awendaw', 'SC')
('28511', 'Atlantic', 'NC')
('71039', 'Heflin', 'LA')
('13034', 'Cayuga', 'NY')
('71037', 'Haughton', 'LA')
('30293', 'Woodbury', 'GA')
('02114', 'Boston', 'MA')
('96120', 'Markleeville', 'CA')
('84636', 'Holden', 'UT')
('65723', 'Pierce City', 'MO')
('31086', 'Smarr', 'GA')
('42716', 'Buffalo', 'KY')
('53713', 'Madison', 'WI')
('99901', 'Ketchikan', 'AK')
('01985', 'West Newbury', 'MA')
('38782', 'Winterville', 'MS')
('73703', 'Enid', 'OK')
('60144

39331

In [27]:
def count_rows_in_table(table_name):
    try:
        conn = duckdb.connect(db_path)
        conn.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = conn.fetchone()[0]
        conn.close()
        return count
    except sqlite3.Error as e:
        print(f"Error counting rows in {table_name}: {e}")
        conn.close()
        return None
count_rows_in_table('DataEntry')

36216155